<a href="https://colab.research.google.com/github/harishvicky-23/GEN-AI/blob/main/(Project)_Novel_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate safetensors
!pip install --upgrade Pillow requests

In [24]:
import torch
from diffusers import StableDiffusionXLPipeline, KDPM2AncestralDiscreteScheduler, AutoencoderKL
from PIL import ImageDraw, ImageFont, Image
import requests
import json
import time

# 🔑 OpenRouter API key
API_KEY = "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"  # <-- Replace this

# 🚀 Load VAE and anime model
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16
)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "enhanceaiteam/AnimeSAI",
    vae=vae,
    torch_dtype=torch.float16
)
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to('cuda')


def generate_story(prompt, title="My Story"):
    print(f"\n📖 Generating full story: {title}")
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a creative story writer. Write vivid and emotional stories with strong character development."},
            {"role": "user", "content": f"Write a complete 1000-word story based on: '{prompt}'."}
        ],
        "temperature": 0.9,
        "max_tokens": 3000
    }

    try:
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, data=json.dumps(data))
        result = response.json()
        if 'choices' not in result:
            print(f"❌ Error generating story:\n{json.dumps(result, indent=2)}")
            story_text = "[Error generating story]"
        else:
            story_text = result['choices'][0]['message']['content']
    except Exception as e:
        print(f"Exception: {e}")
        story_text = "[Exception generating story]"

    filename = f"{title.replace(' ', '_').lower()}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(story_text)
    print(f"\n✅ Story saved as: {filename}")


def generate_image(prompt, title):
    print("\n🖼️ Generating cover image...")
    prompt = f"{prompt}, anime, masterpiece, high quality"
    image = pipe(
        prompt,
        negative_prompt="low quality, blurry",
        width=1024,
        height=1024,
        guidance_scale=7,
        num_inference_steps=50,
        clip_skip=2
    ).images[0]

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    # Center the title at bottom
    bbox = draw.textbbox((0, 0), title, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    W, H = image.size

    draw.text(
        ((W - text_width) / 2, H - text_height - 20),
        title,
        fill="white",
        font=font
    )

    filename = f"{title.replace(' ', '_').lower()}.png"
    image.save(filename)
    print(f"✅ Cover image saved as: {filename}")

# 🧠 MAIN FLOW
if __name__ == "__main__":
    img_prompt = input("Enter prompt for cover pic image: ")
    story_prompt = input("📥 Enter your story idea (1 line): ")
    story_title = input("🎨 Enter a title for your story: ")
    generate_image(img_prompt, story_title)
    generate_story(story_prompt, story_title)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Enter prompt for cover pic image: a 20 year old mid muscular guy, with a one sidded bag around his shoulder, on formal dress with his shirts cuff folded, standing outside anna university, title written as 'DRAGON' floating above the character matching the environment font
📥 Enter your story idea (1 line): a middle bencher high schooler who wins his life from college
🎨 Enter a title for your story: DRAGON'ism

🖼️ Generating cover image...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Cover image saved as: dragon'ism.png

📖 Generating full story: DRAGON'ism

✅ Story saved as: dragon'ism.txt


In [15]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "dreamlike-art/dreamlike-anime-1.0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

#prompt = "anime, masterpiece, high quality, 1man, solo, long hair, looking at viewer, angry, red eyes, warrior dress, muscular, middle of a forest, with a sword, with the title 'never rest' written on sky"
prompt = "a muscular agent, white suit, red eyes, modern weapons, riding on a motorbike, front view, white sports motorbike, with the title 'AGENT AWAKENED' on top right of the image"
negative_prompt = 'simple background, duplicate, retro style, low quality, lowest quality, 1980s, 1990s, 2000s, 2005 2006 2007 2008 2009 2010 2011 2012 2013, bad anatomy, bad proportions, extra digits, lowres, username, artist name, error, duplicate, watermark, signature, text, extra digit, fewer digits, worst quality, jpeg artifacts, blurry'
image = pipe(prompt, negative_prompt=negative_prompt).images[0]

image.save("./result.jpg")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler

model = "Linaqruf/animagine-xl"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
    )

pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to('cpu')

#prompt = "anime, masterpiece, high quality, 1man, solo, long hair, looking at viewer, angry, red eyes, warrior dress, muscular, middle of a forest, with a sword, with the title 'never rest' written on sky"
prompt = "a muscular agent, white suit, red eyes, modern weapons, riding on a motorbike, front view, white sports motorbike, title written as 'AGENT ROSS' floating above the character in matching font"
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"

image = pipe(
    prompt,
    negative_prompt=negative_prompt,
    width=1024,
    height=1024,
    guidance_scale=12,
    target_size=(1024,1024),
    original_size=(4096,4096),
    num_inference_steps=50
    ).images[0]

image.save("anime_girl.png")


"with the title '<Your Title>' written on a sign/scroll/cloud/stone"

"with '<Your Title>' carved into a tree / glowing in the sky"

"title '<Your Title>' floating above the character in matching font"

"the word '<Your Title>' stylized as part of the environment"

Choose font style or medium to match the theme:

"gothic letters", "sci-fi holographic text", "ancient runes", "pixelated game-style font"

Include scene context for where to place text:

"at the top of the mountain", "on a banner in the background", "hovering above the protagonist"